# HHL algorithm for solving linear equations
In the following notebook, we are going to let a real quantum device solve a linear system $A\vec x = \vec b$ .
We will use a quantum algorithm that was proposed by Aram Harrow, Avinatan Hassidim, and Seth Lloyd in 2008.
Very simplified, one can distinguish 4 main steps that need to be performed on our quantum device:
1. Express the solution vector $\vec b$ as a quantum state $|b\rangle$ on a register
2. Use Quantum Phase Estimation (QPE) to decompose $|b\rangle$ into eigenvectors of A and obtain A's eigenvalues
3. Invert the (in the eigenvector base, diagonal) matrix A to prepare the state $A^{-1}|b\rangle$ = $|x\rangle$
4. Apply a linear operator M to $|x\rangle$ to obtain information on the solution vector $\vec x$

Be reminded that in the general case, the entries of $\vec x$ can not be efficiently read out because we would need to know all coefficients describing the quantum state.
In the following example, we ignore this constraint and show for our small linear system as a proof of principle that $\vec x$ is calculated correctly.


As usual, lets get set up.
We import the classes and functions from qiskit as well as some custom functions that will help us adding special gates to the circuit

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, register,get_backend,available_backends
from qiskit.tools.qi.pauli import Pauli
from qiskit_aqua import Operator, QuantumAlgorithm, AlgorithmError
from qiskit_aqua import get_initial_state_instance, get_iqft_instance

from qiskit.tools.visualization._circuit_visualization import matplotlib_circuit_drawer

import numpy as np


from qpe import QPE
from fix_rotation import add_eigenvalue_inversion, add_measurement_gates, generate_matrix, print_linsystem

In [ ]:
try:
    import Qconfig
    qx_config = {
       "APItoken": Qconfig.APItoken,
       "url": Qconfig.config['url']}
except Exception as e:
    print(e)
    qx_config = {
       "APItoken":"6481b0a273869ee834cbd427bbd4480bea29047f6828d578850275ddf012c627b8de551f7297713eb4175791b7c024d11e58e80c47f243b734c4ce427110c3e3",
       "url":"https://quantumexperience.ng.bluemix.net/api"}
    #register(qx_config['APItoken'], qx_config['url'])

In [ ]:
available_backends({'local': False, 'simulator': False})

In [ ]:
N = 2
N_ev = 3
N_sv = int(np.log2(N))
measure = True
backend = "local_qasm_simulator"#"ibmqx5"
print(N,N_ev,N_sv)

## State preparation and QPE

The HHL algorithm starts off with the preparation of $\vec b$ in one register followed by the Quantum Phase Estimation that decomposes $|b\rangle$ into eigenvectors of $A$ and prepares the eigenvalues in a second register.

In [ ]:
matrix =generate_matrix()#np.array([[2, -1 ],[-1, 2]])#generate_matrix()
w, v = np.linalg.eigh(matrix) 

print(matrix)
print("Eigenvalues:", w)

invec =  [1,0]#1/np.sqrt(2)*np.array([1,1])#[1,0]#

params = {
    'algorithm': {
            'name': 'QPE',
            'num_ancillae':2,
            'num_time_slices': 1,
            'expansion_mode': 'trotter',
            'expansion_order': 1,
            'hermitian_matrix': True,
            'negative_evals': False,
            'backend' : backend,#"local_qasm_simulator",
            'evo_time': 2*np.pi/32,
            #'use_basis_gates': False,
    },
    "iqft": {
        "name": "STANDARD"
    },
    "initial_state": {
        "name": "CUSTOM",
        "state_vector": invec#[1/2**0.5,1/2**0.5]
    }
}

In [ ]:
qpe = QPE()
qpe.init_params(params, matrix)

qc = qpe._setup_qpe(measure=False)
matplotlib_circuit_drawer(qc)

Take a look at above circuit. The gate(s) in front of the first U gate that is applied to quibit 0 of the topmost register, prepare us the initial state vector $\vec b$ .
Following, so called Hamiltonian simulation techniques are used to apply the operator $\exp{i At}$ to our register containing the state vector. These


## Inversion and rotation
The last step before we can perform our measurement is to invert $A$. We have $\vec b$ represented in the basis of eigenvectors of A so simply using the inverse eigenvalues gives us $A^{-1}$ . 
Given that this transformation of the eigenvalues from $\lambda$ to $\lambda^{-1}$ is not unitary, we have a probability of failing this step. Unfortunately we do not have direct access to the state of the qubits and we can not "measure" if the inversion suceeded.
Instead we will perform a rotation of $\arcsin{\frac{C}{\lambda}}$ on a "control" qubit. One can show that the state of the control qubit changes from $|0\rangle$ to $\sqrt{1-\frac{C^2}{\lambda_{j}^2}}|0\rangle+\frac{C}{\lambda_j}|1\rangle$ (corresponding to a rotation around the y axis of the Bloch sphere), hence a measurement result of 1 in the control qubit assures us, that the inversion has suceeded.

In [ ]:

add_eigenvalue_inversion(qc)
matplotlib_circuit_drawer(qc)

In [ ]:
qc += qpe._construct_inverse()
matplotlib_circuit_drawer(qc)

Now we add some measurements to our circuit. The qubit storing the solution vector and the control qubit are read out into classical registers

In [ ]:
add_measurement_gates(qc)
matplotlib_circuit_drawer(qc)

We are ready to run our HHL program! 

In [ ]:
result = execute(qc, backend=backend, shots=1024).result()
print(result.get_counts())

In [ ]:
N0 = result.get_counts(qc)['1 1']
N1 = result.get_counts(qc)['0 1']
print(N1/N0)

The ratio of N0 and N1, i.e. the  relative probability of measuring 1, 0 respectively in the solution vector qubit, gives us the relative scale of the two entries in the vector.

Lets repeat our program several times to obtain better statistics... 

In [ ]:
N0 = 0
N1 = 0
for i in range(5):
    qpe = QPE()


    qpe.init_params(params, matrix)

    qc = qpe._setup_qpe(measure=False)
    add_eigenvalue_inversion(qc)
    qc += qpe._construct_inverse()
    add_measurement_gates(qc)
    result = execute(qc, backend=backend, shots=8100).result()
    print(result.get_counts(qc))
    N0 = N0 + result.get_counts(qc)['1 1']
    N1 = N1 + result.get_counts(qc)['0 1']

    

In [ ]:
print_linsystem(matrix,np.sqrt(N1/N0),invec)